#Lab Machine Learning Hyper-parameter Tuning

In the practical machine learning works, it’s very hard to find best parameters - such as learning rate (in neural networks), iterations or epoch,  kernel functions (in svm etc), [regularization parameters], so on and so forth. In Spark machine learning, you can quickly find best parameters by scaling Spark massive workers.

In this lab, we change the source code of Last exercise and find the best values of parameters - "learningRate" and "numLeaves" - for LightGBM classifier in last exercise by grid search. (Here we explore only classification's parameters, but you can also tune transformation's parameters.)

Before starting,

- Install MMLSpark to use LightGBM into your cluster. (See Spark Machine Learning Pipeline".)

> Note : You can also use ```CrossValidator()``` instead of using ```TrainValidationSplit()```, but please be care for training overheads when using ```CrossValidator()```.    
The word "Cross Validation" means : For example, by setting ```numFolds=5``` in ```CrossValidator()```, 4/5 is used for training and 1/5 is for testing, and moreover each pairs are replaced and averaged. As a result, 5 pairs of dataset are used and the training occurs (3 params x 3 params) x 5 pairs = 45 times. 

(See "[ML Tuning: model selection and hyperparameter tuning](https://spark.apache.org/docs/latest/ml-tuning.html)" in official Spark document.)

In [0]:
# Read dataset
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, TimestampType
df = (sqlContext.read.format("csv").
  option("header", "true").
  option("nullValue", "NA").
  option("inferSchema", True).
  load("/FileStore/tables/flight_weather.csv"))

In [0]:
# ARR_DEL15 = 1 if it's canceled.
from pyspark.sql.functions import when
df = df.withColumn("ARR_DEL15", when(df["CANCELLED"] == 1, 1).otherwise(df["ARR_DEL15"]))

In [0]:
# Remove flights if it's diverted.
df = df.filter(df["DIVERTED"] == 0)

In [0]:
# Select required columns
df = df.select(
  "ARR_DEL15",
  "MONTH",
  "DAY_OF_WEEK",
  "UNIQUE_CARRIER",
  "ORIGIN",
  "DEST",
  "CRS_DEP_TIME",
  "CRS_ARR_TIME",
  "RelativeHumidityOrigin",
  "AltimeterOrigin",
  "DryBulbCelsiusOrigin",
  "WindSpeedOrigin",
  "VisibilityOrigin",
  "DewPointCelsiusOrigin",
  "RelativeHumidityDest",
  "AltimeterDest",
  "DryBulbCelsiusDest",
  "WindSpeedDest",
  "VisibilityDest",
  "DewPointCelsiusDest")

In [0]:
# Drop rows with null values
df = df.dropna()

In [0]:
# Convert categorical values to indexer (0, 1, ...)
from pyspark.ml.feature import StringIndexer
uniqueCarrierIndexer = StringIndexer(inputCol="UNIQUE_CARRIER", outputCol="Indexed_UNIQUE_CARRIER").fit(df)
originIndexer = StringIndexer(inputCol="ORIGIN", outputCol="Indexed_ORIGIN").fit(df)
destIndexer = StringIndexer(inputCol="DEST", outputCol="Indexed_DEST").fit(df)
arrDel15Indexer = StringIndexer(inputCol="ARR_DEL15", outputCol="Indexed_ARR_DEL15").fit(df)

In [0]:
# Assemble feature columns
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
  inputCols = [
    "MONTH",
    "DAY_OF_WEEK",
    "Indexed_UNIQUE_CARRIER",
    "Indexed_ORIGIN",
    "Indexed_DEST",
    "CRS_DEP_TIME",
    "CRS_ARR_TIME",
    "RelativeHumidityOrigin",
    "AltimeterOrigin",
    "DryBulbCelsiusOrigin",
    "WindSpeedOrigin",
    "VisibilityOrigin",
    "DewPointCelsiusOrigin",
    "RelativeHumidityDest",
    "AltimeterDest",
    "DryBulbCelsiusDest",
    "WindSpeedDest",
    "VisibilityDest",
    "DewPointCelsiusDest"],
  outputCol = "features")

In [0]:
# Define classifier
from synapse.ml.lightgbm import LightGBMClassifier
#from mmlspark.lightgbm import LightGBMClassifier
classifier = LightGBMClassifier(featuresCol="features", labelCol="ARR_DEL15", numIterations=10)

In [0]:
# Create pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[uniqueCarrierIndexer, originIndexer, destIndexer, arrDel15Indexer, assembler, classifier])

Note : The following execution will take a long time, because of a serial evaluation by default.    
Use ```setParallelism()``` to improve performance.

In [0]:
# We use a ParamGridBuilder to construct a grid of parameters to search over.
# Run pipeline with ParamGridBuilder
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# 3 x 3 = 9 times training occurs

 # learning rate=0.1, (100, 150, 200, 250, 300), accuracy= (0.78, ?, ?, ?, ?, ?) 
# learning rate= 0.3, (100, 150, 200, 250, 300),  accuracy= (0.78, ?, ?, ?, ?, ?) 
# learning rate=0.5, (100, 150, 200, 250, 300),  accuracy= (0.78, ?, ?, ?, ?, ?) 
# learning rate=0.6, (100, 150, 200, 250, 300),  accuracy= (0.78, ?, ?, ?, ?, ?) 



# num leaves=100, learning [0.1, 0.3. 0.5, 0.6] time =[?, ?, ?, ?]/ walltime
# num leaves=150, learning [0.1, 0.3. 0.5, 0.6] time =[?, ?, ?, ?] walltime
# num leaves=200, learning [0.1, 0.3. 0.5, 0.6] time =[?, ?, ?, ?] walltime
# num leaves=250, learning [0.1, 0.3. 0.5, 0.6] time =[?, ?, ?, ?] walltime
# num leaves=3100, learning [0.1, 0.3. 0.5, 0.6] time =[?, ?, ?, ?] walltime

# learning=0.1, [100, 150, 200, 250,300], time =[?, ?, ?, ?,?,?] walltime
# learning=0.6, [100, 150, 200, 250,300], time =[?, ?, ?, ?,?,?]walltime
# learning=0.2, [100, 150, 200, 250,300], time =[?, ?, ?, ?,?,?] walltime

paramGrid = ParamGridBuilder() \
 .addGrid(classifier.learningRate, [0.1, 0.3, 0.5, 0.6]) \
 .addGrid(classifier.numLeaves, [100, 150, 200, 250,300]) \
 .build()

#   TrainValidationSplit creates a single (training, test) dataset pair. It splits the dataset into these two parts using the trainRatio parameter. For example with trainRatio=0.75, TrainValidationSplit will generate a training and test dataset pair where 75% of the data is used for training and 25% for validation. A must reading from https://spark.apache.org/docs/2.0.2/ml-tuning.html
## https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator.metricName
## This is an instance of an Evaluator. It allows to compute the some common measure for classification tasks. It computes precision, recall, f1s for each class, and a global accuracy.

# Set appropriate parallelism by setParallelism() in production
# (It takes a long time)
tvs = TrainValidationSplit(
  estimator=pipeline,
  estimatorParamMaps=paramGrid,
  evaluator=MulticlassClassificationEvaluator(labelCol="ARR_DEL15", predictionCol="prediction"),
  trainRatio=0.8)  # data is separated by 80% and 20%, in which the former is used for training and the later for evaluation
model = tvs.fit(df)

In [0]:
# The hyperparameters as well as the evaluations are exposed by getEstimatorParamMaps and validationMetrics, respectively. They can be combined to display all of the parameter combinations sorted by metric value
# View all results (accuracy) by each params. Read further from here https://www.oreilly.com/library/view/advanced-analytics-with/9781491972946/ch04.html
list(zip(model.validationMetrics, model.getEstimatorParamMaps()))

Out[21]: [(0.7322311897943244,
  {Param(parent='LightGBMClassifier_1c1039c3337d', name='learningRate', doc='Learning rate or shrinkage rate'): 0.1,
   Param(parent='LightGBMClassifier_1c1039c3337d', name='numLeaves', doc='Number of leaves'): 100}),
 (0.7322311897943244,
  {Param(parent='LightGBMClassifier_1c1039c3337d', name='learningRate', doc='Learning rate or shrinkage rate'): 0.1,
   Param(parent='LightGBMClassifier_1c1039c3337d', name='numLeaves', doc='Number of leaves'): 150}),
 (0.7322311897943244,
  {Param(parent='LightGBMClassifier_1c1039c3337d', name='learningRate', doc='Learning rate or shrinkage rate'): 0.1,
   Param(parent='LightGBMClassifier_1c1039c3337d', name='numLeaves', doc='Number of leaves'): 200}),
 (0.7322311897943244,
  {Param(parent='LightGBMClassifier_1c1039c3337d', name='learningRate', doc='Learning rate or shrinkage rate'): 0.1,
   Param(parent='LightGBMClassifier_1c1039c3337d', name='numLeaves', doc='Number of leaves'): 250}),
 (0.7322311897943244,
  {Param(

In [0]:
#method transform(), which converts one DataFrame into another, generally by appending one or more columns.
# Predict using best model
# Use the transform method on the trained model to see its prediction. Now that lrModel is a trained estimator, we can transform data using its .transform() method.v
df10 = df.limit(10)
model.bestModel.transform(df10)\
  .select("ARR_DEL15", "prediction")\
  .show()

+---------+----------+
|ARR_DEL15|prediction|
+---------+----------+
|        0|       0.0|
|        0|       0.0|
|        0|       0.0|
|        1|       1.0|
|        0|       0.0|
|        0|       0.0|
|        0|       0.0|
|        0|       0.0|
|        1|       1.0|
|        0|       0.0|
+---------+----------+

